# Cubes

> Purpose is to provide a **framework** for giving analyst or any application end-user understandable and natural way of reporting using `concept of data Cubes` – multidimensional data objects.

This directory contains following files:

* `model.json`       - logical model
    - Logical model describes the data from user’s or analyst’s perspective: data how they are being measured, aggregated and reported.
    - Model is independent of physical implementation of data. 
    - This physical independence makes it easier to focus on data instead on ways of how to get the data in understandable form.
* `slicer.ini`       - server configuration file
* `BlackFriday.csv`  - source data
* `prepare_data.py`  - script for preparing the data: load them into database and create a view

---
The dimension provides context for facts. Is used to:
- Filter Queries
- controls scope of aggregation of facts
---


References
----------
- https://pythonhosted.org/cubes/server.html
- https://apandre.wordpress.com/data/datacube/
- https://cubes.readthedocs.io/en/v1.0.1/introduction.html

In [129]:
import cubes
import pandas as pd

In [132]:
df = pd.read_csv("BlackFriday.csv")
df.head(5)

,Age,Occupation,Product_Category,Purchase
0,0-17,10,3,8370
1,0-17,10,1,15200
2,0-17,10,12,1422
3,0-17,10,12,1057
4,55+,16,8,7969


# Describing Data Cube

## Dimension 
- `Age` - Discritisized
- `Occupation` - Nominal attribute (represented as integer)
- `Product_Category` - Nominal attribute (represented as integer)

## Measure
- `Purchase` - purchase amount (number)

![Cube](DataCube.jpg)

![overview](https://cubes.readthedocs.io/en/v1.0.1/_images/cubes-analytical-workspace-overview1.png)

In [121]:
def pretty_print(browser, dimension, **kwargs):
    cell = kwargs.get('cell', None)
    count = 0
    if cell:
        for row in browser.aggregate(cell, drilldown=[dimension]).table_rows(dimension):
            print("{:>8s} >>> {}".format(row.label, row.record))
            count += 1
    else:
        for row in browser.aggregate(drilldown=[dimension]).table_rows(dimension):
            print("{:>8s} >>> {}".format(row.label, row.record))
            count += 1
    print("{}\nTotal slices in dimension: {} are {}".format("="*50, dimension, count))

In [122]:
workspace = cubes.Workspace()
workspace.register_default_store("sql", url="sqlite:///data.sqlite")
workspace.import_model("model.json")

### Browser
Core of the Cubes analytics functionality is the aggregation browser. 

The browser module contains utility classes and functions for the browser to work.

In [123]:
browser = workspace.browser('black_friday')

In [124]:
pretty_print(browser, dimension="Age")

    0-17 >>> {'Purchase_sum': 302980, 'record_count': 29, 'Age': '0-17'}
   18-25 >>> {'Purchase_sum': 880881, 'record_count': 94, 'Age': '18-25'}
   26-35 >>> {'Purchase_sum': 1756847, 'record_count': 181, 'Age': '26-35'}
   36-45 >>> {'Purchase_sum': 843641, 'record_count': 77, 'Age': '36-45'}
   46-50 >>> {'Purchase_sum': 509699, 'record_count': 66, 'Age': '46-50'}
   51-55 >>> {'Purchase_sum': 205344, 'record_count': 29, 'Age': '51-55'}
     55+ >>> {'Purchase_sum': 174551, 'record_count': 24, 'Age': '55+'}
Total slices in dimension: Age are 7


In [125]:
pretty_print(browser, dimension="Occupation")

       0 >>> {'Purchase_sum': 654409, 'record_count': 60, 'Occupation': '0'}
       1 >>> {'Purchase_sum': 645130, 'record_count': 76, 'Occupation': '1'}
      10 >>> {'Purchase_sum': 302980, 'record_count': 29, 'Occupation': '10'}
      11 >>> {'Purchase_sum': 58665, 'record_count': 5, 'Occupation': '11'}
      12 >>> {'Purchase_sum': 262402, 'record_count': 21, 'Occupation': '12'}
      13 >>> {'Purchase_sum': 100218, 'record_count': 14, 'Occupation': '13'}
      14 >>> {'Purchase_sum': 72459, 'record_count': 9, 'Occupation': '14'}
      15 >>> {'Purchase_sum': 131481, 'record_count': 15, 'Occupation': '15'}
      16 >>> {'Purchase_sum': 52767, 'record_count': 12, 'Occupation': '16'}
      17 >>> {'Purchase_sum': 179909, 'record_count': 20, 'Occupation': '17'}
      18 >>> {'Purchase_sum': 8652, 'record_count': 1, 'Occupation': '18'}
      19 >>> {'Purchase_sum': 40856, 'record_count': 7, 'Occupation': '19'}
       2 >>> {'Purchase_sum': 216111, 'record_count': 20, 'Occupation': '2'}

In [126]:
pretty_print(browser, dimension="Product_Category")

       1 >>> {'Purchase_sum': 1885384, 'record_count': 134, 'Product_Category': '1'}
      10 >>> {'Purchase_sum': 47395, 'record_count': 2, 'Product_Category': '10'}
      11 >>> {'Purchase_sum': 62458, 'record_count': 13, 'Product_Category': '11'}
      12 >>> {'Purchase_sum': 10181, 'record_count': 7, 'Product_Category': '12'}
      13 >>> {'Purchase_sum': 2084, 'record_count': 3, 'Product_Category': '13'}
      14 >>> {'Purchase_sum': 23551, 'record_count': 3, 'Product_Category': '14'}
      15 >>> {'Purchase_sum': 51850, 'record_count': 4, 'Product_Category': '15'}
      16 >>> {'Purchase_sum': 104010, 'record_count': 6, 'Product_Category': '16'}
      18 >>> {'Purchase_sum': 3794, 'record_count': 1, 'Product_Category': '18'}
       2 >>> {'Purchase_sum': 240144, 'record_count': 18, 'Product_Category': '2'}
       3 >>> {'Purchase_sum': 289368, 'record_count': 27, 'Product_Category': '3'}
       4 >>> {'Purchase_sum': 26976, 'record_count': 13, 'Product_Category': '4'}
       5 >>

In [100]:
# Aggregations

In [115]:
result = browser.aggregate()
result.summary

{'Purchase_sum': 4673943, 'record_count': 500}

In [145]:
cuts = [
    cubes.PointCut("Age", ["0-17"]),
    cubes.PointCut("Occupation", ["10"])
]
cell = cubes.Cell(browser.cube, cuts)
result = browser.aggregate(cell)
result.summary

{'Purchase_sum': 302980, 'record_count': 29}